In [2]:
import pickle
import json
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

sns.set(style='ticks', font_scale = 1.5)
sns.set(rc={'figure.figsize': (18, 6)})
sns.set_style('whitegrid')

plt.rcParams.update({'font.size': 12})

models = ["dbscan", "hdbscan", "birch", "kmeans-minibatch"]
norm_models = ["dbscan", "hdbscan"]
suite = 'generic'
num_datasets = 56
df_ref = pickle.load(open('generic_dfs.pkl', 'rb'))


In [3]:
from fanova import fANOVA

import ConfigSpace
from ConfigSpace.hyperparameters import CategoricalHyperparameter, UniformFloatHyperparameter, \
    UniformIntegerHyperparameter, Constant
from hydra import compose, initialize
from omegaconf import OmegaConf

from clustering_hyperparameters.utils.type_utils import get_type_from_str
metadata_cols = ["adjusted_rand_score", 
                "trial_status",
                "generator_model",
                "trial_index",
                "arm_name",
                "compute_time"]

res_df = { m : pd.DataFrame() for m in models } 
for model in models:
    all_df_model = pd.concat([df_ref['random'][model], df_ref['bayesian'][model]])
    all_df_model_groups = all_df_model.groupby('dataset')
    

    with initialize(config_path="../src/clustering_hyperparameters/conf", job_name="app"):
        for dataset_index in range(num_datasets):
            cfg = compose(config_name="config", overrides=["model="+model, "suite="+suite, "dataset_index="+str(dataset_index)])
            cfg.dataset_index = str(cfg.dataset_index)
            cfg = OmegaConf.to_container(cfg, resolve=True)
            dataset = cfg["suite"]["datasets"][dataset_index]["name"]
            dataset_with_norm = dataset 
            if model not in norm_models:
                dataset_with_norm += "_normalized"
            try :
                dataset_df = all_df_model_groups.get_group(dataset_with_norm)
            except KeyError as e:
                continue
            print("model:", model, "dataset_index:", dataset_index, "dataset", dataset)
            
            
            params = [ x for x in cfg["model"]["params"] if x['type']=='range'] 
            param_names = [ x['name'] for x in params ]
            cs = ConfigSpace.ConfigurationSpace()
            for ind, param in enumerate(params):
                log_scale = param['log_scale'] if 'log_scale' in param else False
                if param['value_type'] == "int":
                    cs.add_hyperparameter(UniformIntegerHyperparameter(param["name"], lower=param['bounds'][0], upper=param['bounds'][1], log=log_scale))
                elif param['value_type'] == "float":
                    cs.add_hyperparameter(UniformFloatHyperparameter(param["name"], lower=param['bounds'][0], upper=param['bounds'][1], log=log_scale))
                elif param['value_type'] == "str":
                    cs.add_hyperparameter(CategoricalHyperparameter(param["name"], choices=param['choices']))
                    
            X , Y = dataset_df[param_names], dataset_df['adjusted_rand_score'].to_numpy()
            f = fANOVA(X = X, Y = Y, config_space=cs, seed=0)
            res_dict = {'dataset': dataset}
            for param in param_names: 
                try:
                    res_dict[param] = f.quantify_importance((param,))[(param,)]['individual importance']
                except Exception:
                    res_dict[param] = 0.0
            res_df[model] = res_df[model].append(res_dict, ignore_index=True)  


/opt/homebrew/anaconda3/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


model: dbscan dataset_index: 0 dataset kr-vs-kp
model: dbscan dataset_index: 1 dataset letter
model: dbscan dataset_index: 2 dataset balance-scale
model: dbscan dataset_index: 3 dataset mfeat-factors
model: dbscan dataset_index: 4 dataset mfeat-fourier
model: dbscan dataset_index: 5 dataset mfeat-karhunen
model: dbscan dataset_index: 6 dataset mfeat-morphological
model: dbscan dataset_index: 7 dataset mfeat-zernike
model: dbscan dataset_index: 8 dataset cmc
model: dbscan dataset_index: 9 dataset credit-g
model: dbscan dataset_index: 10 dataset diabetes
model: dbscan dataset_index: 11 dataset spambase
model: dbscan dataset_index: 12 dataset splice
model: dbscan dataset_index: 13 dataset tic-tac-toe
model: dbscan dataset_index: 14 dataset vehicle
model: dbscan dataset_index: 15 dataset electricity
model: dbscan dataset_index: 16 dataset satimage
model: dbscan dataset_index: 17 dataset vowel
model: dbscan dataset_index: 18 dataset isolet
model: dbscan dataset_index: 19 dataset analcatdata

In [5]:
from IPython.display import display
display()
all_dfs = []
for model in models:
    df = res_df[model].copy()
    df.set_index(['dataset'], inplace=True)
    df = df.T.copy()
    df['model'] = [model] * len(df)
    df.index.name = 'hyperparameters'
    df.reset_index(level=0, inplace=True)
    df.set_index(['model', 'hyperparameters'], inplace=True)
    all_dfs.append(df)

all_dfs = pd.concat(all_dfs)
all_dfs = all_dfs * 100
display(all_dfs)
with open('importances_generic.tex', 'w') as f:
    f.write(all_dfs.to_latex(float_format="%.3f"))

dataset                                      kr-vs-kp     letter  \
model            hyperparameters                                   
dbscan           eps                        41.220710  63.467530   
                 min_samples                 6.056756   6.106907   
hdbscan          cluster_selection_epsilon   0.847365        NaN   
                 min_cluster_size           19.361229        NaN   
                 min_samples                29.355308        NaN   
birch            branching_factor           10.670878  10.377799   
                 threshold                  37.780271  45.022856   
kmeans-minibatch batch_size                  0.357834   1.163492   
                 init_size                  27.138047   4.020237   
                 max_iter                    5.542125   3.842833   
                 max_no_improvement          0.826018   3.675254   
                 n_init                     17.901122   4.419958   
                 reassignment_ratio          0.714633   3.690772   
                 tol                         1.243147   3.790358   

dataset                                     balance-scale  mfeat-factors  \
model            hyperparameters                                           
dbscan           eps                            43.121513      69.613667   
                 min_samples                    15.530520       4.482135   
hdbscan          cluster_selection_epsilon       7.681203      28.989496   
                 min_cluster_size               10.553141       3.426232   
                 min_samples                    26.680713       8.786730   
birch            branching_factor               17.866620      16.600725   
                 threshold                      16.064394      38.854121   
kmeans-minibatch batch_size                      1.889769       2.260817   
                 init_size                       8.233833       8.114177   
                 max_iter                        3.346068       2.454882   
                 max_no_improvement              3.430600       1.864339   
                 n_init                          7.676657      13.306405   
                 reassignment_ratio              1.999776       1.324519   
                 tol                             2.303887       3.957199   

dataset                                     mfeat-fourier  mfeat-karhunen  \
model            hyperparameters                                            
dbscan           eps                            77.576333       44.263242   
                 min_samples                     1.776517        6.235089   
hdbscan          cluster_selection_epsilon      10.592440        3.193991   
                 min_cluster_size                7.273913        1.035273   
                 min_samples                    34.797058       70.762848   
birch            branching_factor                7.124044        4.644191   
                 threshold                      60.058822       72.860880   
kmeans-minibatch batch_size                      1.511920       14.141208   
                 init_size                      11.050072        2.824963   
                 max_iter                        2.573653        2.472150   
                 max_no_improvement              4.103923        2.758711   
                 n_init                          4.179897        9.106866   
                 reassignment_ratio              2.775153        1.832510   
                 tol                             4.747563        2.779593   

dataset                                     mfeat-morphological  \
model            hyperparameters                                  
dbscan           eps                                  87.021365   
                 min_samples                           3.645320   
hdbscan          cluster_selection_epsilon            76.879312   
                 min_cluster_size                      0.799200   
                 min_samples                           4.50